### Graph
`g1 = tf.Graph()` <br />
`with g1.as_default():`<br />
### Session
`with tf.Session(Graph=g1) as sess1:`<br />
### variable scope
`with tf.variable_scope("foo"):`<br />
`t = tf.get_variable("v",[1])`<br />
`tf.get_variable_scope()`<br />
`tf.get_variable_scope().reuse_variables()`<br />

In [5]:
import tensorflow as tf
from PIL import Image
import numpy as np

tf.reset_default_graph()
image1 = tf.cast(tf.constant(np.array(Image.open('cameraman.png')).reshape([-1,256,256,1])),tf.float32)/255.0
image2 = tf.cast(tf.constant(np.array(Image.open('emoji_1.png').convert('L')).reshape([-1,429,705,1])),tf.float32)/255.0
def conv_relu(input, kernel_shape, bias_shape):
    # Create variable named "weights".
    weights = tf.get_variable("weights", kernel_shape,
        initializer=tf.random_normal_initializer())
    # Create variable named "biases".
    biases = tf.get_variable("biases", bias_shape,
        initializer=tf.constant_initializer(0.0))
    conv = tf.nn.conv2d(input, weights,
        strides=[1, 1, 1, 1], padding='SAME')
    return tf.nn.relu(conv + biases)

def my_image_filter(input_images):
    with tf.variable_scope("conv1"):
        # Variables created here will be named "conv1/weights", "conv1/biases".
        relu1 = conv_relu(input_images, [5, 5, 1, 32], [32])
    with tf.variable_scope("conv2"):
        # Variables created here will be named "conv2/weights", "conv2/biases".
        return conv_relu(relu1, [5, 5, 32, 32], [32])
    
with tf.variable_scope("hello") as scope:
    #scope.reuse_variables() 在此位置加这句则不能用tf.get_variable()初始化
    result1 = my_image_filter(image1)
    scope.reuse_variables() #表示后面是复用
    result2 = my_image_filter(image2)
# Raises ValueError(... conv1/weights already exists ...)
# 这里一共有4个变量而不是8个,hello/[conv1,conv2]/[weights,biases],达到了共享参数的效果

In [86]:
# name scope 可以累积
with tf.variable_scope("foo"):
    with tf.variable_scope("bar"):
        t = tf.get_variable("v",[1])
print(t.name)

foo/bar/v:0


In [2]:
import tensorflow as tf
g1 = tf.Graph() # 定义新图 
with g1.as_default(): 
    c1 = tf.constant([1.0])
with tf.Graph().as_default() as g2:
    c2 = tf.constant([2.0])

with tf.Session(graph=g1) as sess1:
    print(sess1.run(c1))
with tf.Session(graph=g2) as sess2:
    print(sess2.run(c2))


[ 1.]
[ 2.]
